In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
# Importar datos
df_donantesi = pd.read_csv('./Archivos_Cliente/Base_donantes.csv', encoding='latin-1')
df_donantesi.set_index('PSN', inplace=True)
df_donantesi.head()

C:\Users\ZU956KH\AppData\Local\Temp\ipykernel_9004\1927339027.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_donantesi = pd.read_csv('./Archivos_Cliente/Base_donantes.csv', encoding='latin-1')


,Tipo de registro del contacto,Fecha de nacimiento,Fecha Aniversario Pago,Edad,Fecha de Captación,Monto Actual,Género,Estado Civil,Cantidad de Hijos,Tiene hijos,Ocupación,Churn Probability,Lapsed Probability,RFM Segmento Actual,Otra Clasificación RFM Actual,Cantidad Cuotas Pagadas Global,Cantidad Cuotas No Pagadas Global,Campaña Inicial: Nombre,Donante Activo
PSN,,,,,,,,,,,,,,,,,,,
1020438646,Donante,1/12/1994,NaN,29.0,10/26/2018,NaN,Femenino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,0
1020276703,Donante,11/7/1990,Nº de error,32.0,12/29/2015,20000.0,Femenino,Soltero,NaN,No,Tecnólogo,NaN,NaN,Otra Clasificación,Extraviados,12,80,SPIF F2F 2016,0
1020490284,Donante,9/29/1994,3/1/2023,28.0,2/18/2023,30000.0,Femenino,NaN,1,Si,NaN,0.56,51.0,Otra Clasificación,Constantes,3,0,COF2FR.F.2023,0
1020184782,Donante,8/13/1973,10/31/2023,49.0,6/24/2015,20000.0,Masculino,Soltero,1,Si,NaN,NaN,NaN,Otra Clasificación,Extraviados,12,12,SPIF F2F 2015,0
1020476125,Donante,9/6/1986,NaN,36.0,4/13/2022,30000.0,Femenino,Soltero,2,Si,Asesor de gestión,NaN,NaN,NaN,NaN,0,2,COWCB R.F.2022,0


#### Preprocesamiento de datos

In [4]:
# Remover los registros que tienen "N° de error" en la columna Fecha Aniversario Pago
df_donantesi = df_donantesi[df_donantesi['Fecha Aniversario Pago']!='Nº de error']
# Remover los registros inactivos
df_donantesi = df_donantesi[df_donantesi['Donante Activo']==1]
# Remover registros menores de edad
df_donantesi = df_donantesi[df_donantesi['Edad']>=18]
# Remover registros mayores de 90 años
df_donantesi = df_donantesi[df_donantesi['Edad']<=90]
# Reemplazar textos de la cantidad de hijos
df_donantesi['Cantidad de Hijos'] = df_donantesi['Cantidad de Hijos'].replace('mas de 10', 10)
df_donantesi['Cantidad de Hijos'] = df_donantesi['Cantidad de Hijos'].astype(float)
# Llenar faltantes de tiene hijos
df_donantesi['Tiene hijos'] = np.where(df_donantesi['Cantidad de Hijos']>0.0, 'Si', 'No')
# Llenar faltantes de cantidad de hijos
df_donantesi['Cantidad de Hijos'] = np.where(df_donantesi['Tiene hijos']=='No', 0, df_donantesi['Cantidad de Hijos'])
# Cambiar tipos de dato
df_donantesi['Fecha de Captación'] = pd.to_datetime(df_donantesi['Fecha de Captación'])
df_donantesi['Tipo de registro del contacto'] = df_donantesi['Tipo de registro del contacto'].astype('category')
df_donantesi['Fecha de nacimiento'] = pd.to_datetime(df_donantesi['Fecha de nacimiento'])
df_donantesi['Fecha Aniversario Pago'] = pd.to_datetime(df_donantesi['Fecha Aniversario Pago'])
df_donantesi['Género'] = df_donantesi['Género'].astype('category')
df_donantesi['Estado Civil'] = df_donantesi['Estado Civil'].astype('category')
df_donantesi['Tiene hijos'] = df_donantesi['Tiene hijos'].astype('category')
# Reemplazar NaNs en Fecha Aniversario Pago con la Fecha de Captacion
df_donantesi['Fecha Aniversario Pago'] = np.where(df_donantesi['Fecha Aniversario Pago'].isna(), df_donantesi['Fecha de Captación'], df_donantesi['Fecha Aniversario Pago'])
# Cambiar el año de Fecha Aniversario Pago por 2023
df_donantesi['Fecha Aniversario Pago'] = df_donantesi['Fecha Aniversario Pago'].apply(lambda x: x.replace(year=2023))
# Obtener mes de la fecha actual
today = dt.datetime.now().month
# Remover registros con menos de 3 meses de fecha de captación
df_donantesi = df_donantesi[df_donantesi['Fecha de Captación']<dt.datetime(2023, today-3, 1) - dt.timedelta(days=90)]
# Corregir Churn Probability
df_donantesi['Churn Probability'] = np.where(df_donantesi['Churn Probability']>1, df_donantesi['Churn Probability']/100, df_donantesi['Churn Probability'])
# Corregir Lapsed Probability
df_donantesi['Lapsed Probability'] = np.where(df_donantesi['Lapsed Probability']>1, df_donantesi['Lapsed Probability']/100, df_donantesi['Lapsed Probability'])

In [5]:
# Eliminar registros sin Churn Probability
df_donantesi['Churn Probability'].dropna(inplace=True)

# Agregar lifespan de cada donante
df_donantesi['Lifespan'] = df_donantesi['Churn Probability'].apply(lambda x: 1/x)

In [6]:
df_donantesi['Lifespan'].describe()

count    22899.000000
mean        18.402555
std         28.271439
min          1.000000
25%          1.754386
50%          2.941176
75%         29.940120
max         99.009901
Name: Lifespan, dtype: float64

In [7]:
df_donantesi['New Churn'] = df_donantesi[['Churn Probability', 'Lapsed Probability']].max(axis=1)
df_donantesi['New Churn'].describe()

count    22899.000000
mean         0.419245
std          0.269788
min          0.010100
25%          0.260000
50%          0.380000
75%          0.610000
max          1.000000
Name: New Churn, dtype: float64

In [8]:
df_donantesi.head()

,Tipo de registro del contacto,Fecha de nacimiento,Fecha Aniversario Pago,Edad,Fecha de Captación,Monto Actual,Género,Estado Civil,Cantidad de Hijos,Tiene hijos,...,Churn Probability,Lapsed Probability,RFM Segmento Actual,Otra Clasificación RFM Actual,Cantidad Cuotas Pagadas Global,Cantidad Cuotas No Pagadas Global,Campaña Inicial: Nombre,Donante Activo,Lifespan,New Churn
PSN,,,,,,,,,,,,,,,,,,,,,
1020456818,Donante,1964-06-16,2023-07-01,59.0,2021-06-19,30000.0,Masculino,Casado,0.0,No,...,0.59,0.0,Otra Clasificación,Constantes,26,0,COF2FR.F. 2021,1,1.694915,0.59
1020483942,Donante,1975-08-28,2023-10-01,47.0,2022-10-01,50000.0,Femenino,Casado,1.0,Si,...,0.78,0.0,Otra Clasificación,Constantes,10,0,COWCB R.F.2022,1,1.282051,0.78
1020276927,Donante,1941-09-23,2023-01-31,81.0,2015-12-29,20000.0,Masculino,Casado,2.0,Si,...,0.53,0.0,Otra Clasificación,Constantes,81,10,SPIF F2F 2016,1,1.886792,0.53
1020476173,Donante,1988-10-28,2023-04-18,34.0,2022-04-17,30000.0,Femenino,Soltero,0.0,No,...,0.95,0.0,Otra Clasificación,Constantes,16,0,COF2FR.F.2022,1,1.052632,0.95
1020483943,Donante,1982-04-22,2023-10-01,41.0,2022-10-01,30000.0,Femenino,Casado,2.0,Si,...,0.92,0.0,Otra Clasificación,Constantes,10,0,COWCB R.F.2022,1,1.086957,0.92


#### Transacciones

In [9]:
# Importar datos de transacciones individuales
df_transi = pd.read_csv('./Archivos_Cliente/Transacciones_Individuales.csv', sep=';', encoding='latin-1')

In [10]:
# Cambiar tipos de dato
df_transi['Fecha de Donación'] = pd.to_datetime(df_transi['Fecha de Donación'])
df_transi['Etapa'] = df_transi['Etapa'].astype('category')
df_transi['Importe'] = df_transi['Importe'].str.replace(',','.')
df_transi['Importe'] = df_transi['Importe'].astype(float)
df_transi['Canal de la Campaña'] = df_transi['Canal de la Campaña'].astype('category')
df_transi['Tipo de registro'] = df_transi['Tipo de registro'].astype('category')
df_transi['Medio de Pago'] = df_transi['Medio de Pago'].astype('category')
df_transi['Tipo de Compromiso'] = df_transi['Tipo de Compromiso'].astype('category')
#df_transi['Fecha de Registro Contable'] = pd.to_datetime(df_transi['Fecha de Registro Contable'], format='mixed')
df_transi['Fecha efectiva de primer cobro'] = pd.to_datetime(df_transi['Fecha efectiva de primer cobro'])
#df_transi['Fecha de última donación'] = pd.to_datetime(df_transi['Fecha de última donación'], format='mixed')
df_transi['Estado/Tipo'] = df_transi['Estado/Tipo'].astype('category')

C:\Users\ZU956KH\AppData\Local\Temp\ipykernel_9004\3222432061.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_transi['Fecha de Donación'] = pd.to_datetime(df_transi['Fecha de Donación'])
C:\Users\ZU956KH\AppData\Local\Temp\ipykernel_9004\3222432061.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_transi['Fecha efectiva de primer cobro'] = pd.to_datetime(df_transi['Fecha efectiva de primer cobro'])


In [11]:
# Filtrar transacciones efectivas
df_trans_cobradas = df_transi[df_transi['Etapa']=='Cobrada']

In [12]:
df_trans_cobradas['Mes Donación'] = df_trans_cobradas['Fecha de Donación'].dt.month
df_trans_cobradas['Anio Donación'] = df_trans_cobradas['Fecha de Donación'].dt.year

C:\Users\ZU956KH\AppData\Local\Temp\ipykernel_9004\1893558933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_cobradas['Mes Donación'] = df_trans_cobradas['Fecha de Donación'].dt.month
C:\Users\ZU956KH\AppData\Local\Temp\ipykernel_9004\1893558933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_cobradas['Anio Donación'] = df_trans_cobradas['Fecha de Donación'].dt.year


In [13]:
# Tabla de donaciones anuales por donante
donaciones_anuales = pd.pivot_table(df_trans_cobradas, values='Importe', index='PSN', columns=['Anio Donación'], aggfunc='sum', fill_value=0)

# Agregar columna de antiguedad de la primera transaccion
donaciones_anuales.loc[donaciones_anuales[2021]>0, 'Ant_Primer_Transaccion'] = 3
donaciones_anuales.loc[(donaciones_anuales[2022]>0) & (donaciones_anuales[2021]==0), 'Ant_Primer_Transaccion'] = 2
donaciones_anuales['Ant_Primer_Transaccion'].fillna(1, inplace=True)

# Tasa de descuento promedio
ipc_2022 = 0.0694
ipc_2023 = 0.1325
ipc = (ipc_2022 + ipc_2023)/2

# Cálcular montos anuales en valor presente
donaciones_anuales['Monto_t-2_presente'] = donaciones_anuales.apply(lambda x:x[2021]*(1-ipc)**(x['Ant_Primer_Transaccion']-1), axis=1)
donaciones_anuales['Monto_t-1_presente'] = donaciones_anuales.apply(lambda x:x[2022]*(1-ipc)**(x['Ant_Primer_Transaccion']-1), axis=1)

# Sumar todos los montos
donaciones_anuales['Valor_Total'] = donaciones_anuales['Monto_t-2_presente'] + donaciones_anuales['Monto_t-1_presente'] + donaciones_anuales[2023]

# Obtener monto promedio anual en valor presente
donaciones_anuales['Total_Anual'] = donaciones_anuales['Valor_Total'] / donaciones_anuales['Ant_Primer_Transaccion']

# Agregar campo de lifespan
donaciones_anuales = pd.merge(donaciones_anuales, df_donantesi['Lifespan'], on='PSN', how='inner')
donaciones_anuales['Lifespan'] = round(donaciones_anuales['Lifespan'], 0)
donaciones_anuales.dropna(how='any', inplace=True)
donaciones_anuales['Lifespan'] = donaciones_anuales['Lifespan'].astype(int)

# Definición de función para calcular el valor presente
def calcular_vp(row):
    montos = [row['Total_Anual']/((1+ipc)**t) for t in range(1, int(row['Lifespan']+1))]
    return sum(montos)


# Calcular el DLTV
donaciones_anuales['DLTV'] = donaciones_anuales.apply(calcular_vp, axis=1)


In [14]:
donaciones_anuales.head()

,2021,2022,2023,Ant_Primer_Transaccion,Monto_t-2_presente,Monto_t-1_presente,Valor_Total,Total_Anual,Lifespan,DLTV
PSN,,,,,,,,,,
1020000012,540000.0,720000.0,420000,3.0,436477.08735,581969.449800,1.438447e+06,479482.179050,6,2.082469e+06
1020000038,600000.0,550000.0,400000,3.0,484974.54150,444559.996375,1.329535e+06,443178.179292,5,1.675925e+06
1020000079,960000.0,960000.0,560000,3.0,775959.26640,775959.266400,2.111919e+06,703972.844267,6,3.057468e+06
1020000087,300000.0,330000.0,300000,3.0,242487.27075,266735.997825,8.092233e+05,269741.089525,7,1.309115e+06
1020000095,960000.0,960000.0,80000,3.0,775959.26640,775959.266400,1.631919e+06,543972.844267,56,5.363849e+06


In [16]:
scaler = StandardScaler()
dltv_scaled = scaler.fit_transform(donaciones_anuales)
kmeans = KMeans(n_clusters=4, random_state=123, n_init=10)
kmeans.fit(dltv_scaled)
print(kmeans.labels_)

C:\Users\ZU956KH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\ZU956KH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


[2 2 2 ... 0 0 0]


In [ ]:
# Pintar clusters vs DLTV
